# Urban Retail Co. - Inventory Forecasting and Analysis

**Project:** Solving Inventory Inefficiencies Using Advanced SQL Analytics with Python Integration

**Objective:** Integrate Python forecasting models with SQL data to provide predictive inventory insights and optimize stock levels.

## 1. Setup and Database Connection

First, we'll import necessary libraries and establish a connection to our SQL database.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Database connection parameters
db_params = {
    'host': 'localhost',
    'database': 'inventory_db',
    'user': 'inventory_user',
    'password': 'secure_password'
}

# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}/{db_params['database']}")

print("Database connection established.")

## 2. Data Extraction from SQL Views

We'll leverage the SQL views we created to extract the data needed for our forecasting models.

In [ ]:
# Query to extract sales trend data from our view
sales_trend_query = """
SELECT * FROM solving_inventory.vw_sales_trend_analysis
WHERE month_start >= CURRENT_DATE - INTERVAL '24 months'
ORDER BY month_start, store_id, product_id
"""

# Load data into pandas DataFrame
sales_trend_df = pd.read_sql(sales_trend_query, engine)

# Display the first few rows
print("Sales Trend Data Sample:")
print(sales_trend_df.head())

# Get inventory data
inventory_query = """
SELECT * FROM solving_inventory.vw_inventory_kpi_dashboard
ORDER BY region, store_id, category_id
"""

inventory_df = pd.read_sql(inventory_query, engine)

# Display the first few rows
print("\nInventory KPI Data Sample:")
print(inventory_df.head())

## 3. Data Preprocessing for Forecasting

We'll prepare the data for time series analysis and forecasting.

In [ ]:
# Function to prepare time series data for a specific product at a specific store
def prepare_time_series(store_id, product_id):
    # Filter data for the specific store and product
    product_data = sales_trend_df[(sales_trend_df['store_id'] == store_id) & 
                                 (sales_trend_df['product_id'] == product_id)]
    
    # Sort by date
    product_data = product_data.sort_values('month_start')
    
    # Set month_start as index
    product_data = product_data.set_index('month_start')
    
    # Return the monthly units sold as a time series
    return product_data['monthly_units_sold']

# Get a list of top-selling products
top_products_query = """
SELECT store_id, product_id, SUM(monthly_units_sold) as total_units
FROM solving_inventory.vw_sales_trend_analysis
GROUP BY store_id, product_id
ORDER BY total_units DESC
LIMIT 10
"""

top_products = pd.read_sql(top_products_query, engine)
print("Top Selling Products:")
print(top_products.head())

# Select the top product for demonstration
sample_store_id = top_products.iloc[0]['store_id']
sample_product_id = top_products.iloc[0]['product_id']

# Prepare time series data
ts_data = prepare_time_series(sample_store_id, sample_product_id)

print(f"\nTime Series Data for Product {sample_product_id} at Store {sample_store_id}:")
print(ts_data.head())

# Plot the time series
plt.figure(figsize=(12, 6))
ts_data.plot()
plt.title(f"Monthly Sales for Product {sample_product_id} at Store {sample_store_id}")
plt.xlabel("Month")
plt.ylabel("Units Sold")
plt.tight_layout()
plt.savefig("sample_product_sales.png")
plt.close()

## 4. Time Series Decomposition

We'll decompose the time series to understand its trend, seasonality, and residual components.

In [ ]:
# Ensure we have enough data points
if len(ts_data) >= 12:  # Need at least 12 months for yearly seasonality
    # Decompose the time series
    decomposition = seasonal_decompose(ts_data, model='multiplicative', period=12)
    
    # Plot the decomposition
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 10))
    
    decomposition.observed.plot(ax=ax1)
    ax1.set_title('Observed')
    
    decomposition.trend.plot(ax=ax2)
    ax2.set_title('Trend')
    
    decomposition.seasonal.plot(ax=ax3)
    ax3.set_title('Seasonality')
    
    decomposition.resid.plot(ax=ax4)
    ax4.set_title('Residuals')
    
    plt.tight_layout()
    plt.savefig("time_series_decomposition.png")
    plt.close()
    
    print("Time series decomposition completed and saved.")
else:
    print("Not enough data points for seasonal decomposition. Need at least 12 months.")

## 5. ARIMA Forecasting Model

We'll implement an ARIMA model to forecast future sales.

In [ ]:
# Function to fit ARIMA model and forecast
def fit_arima_and_forecast(time_series, forecast_periods=3):
    # Split data into train and test sets
    train_size = int(len(time_series) * 0.8)
    train, test = time_series[:train_size], time_series[train_size:]
    
    # Fit ARIMA model - parameters (p,d,q) would ideally be determined through analysis
    model = ARIMA(train, order=(1, 1, 1))
    model_fit = model.fit()
    
    # Forecast for test period
    forecast = model_fit.forecast(steps=len(test))
    
    # Calculate error
    mse = mean_squared_error(test, forecast)
    rmse = np.sqrt(mse)
    
    # Forecast future periods
    future_forecast = model_fit.forecast(steps=forecast_periods)
    
    return {
        'train': train,
        'test': test,
        'forecast': forecast,
        'future_forecast': future_forecast,
        'rmse': rmse,
        'model_summary': model_fit.summary()
    }

# Apply the function to our sample data
if len(ts_data) > 5:  # Ensure we have enough data
    forecast_results = fit_arima_and_forecast(ts_data)
    
    # Print results
    print(f"ARIMA Model RMSE: {forecast_results['rmse']:.2f}")
    print("\nFuture Forecast (Next 3 Months):")
    print(forecast_results['future_forecast'])
    
    # Plot results
    plt.figure(figsize=(12, 6))
    
    # Plot training data
    plt.plot(forecast_results['train'].index, forecast_results['train'].values, label='Training Data')
    
    # Plot test data
    plt.plot(forecast_results['test'].index, forecast_results['test'].values, label='Test Data')
    
    # Plot forecast for test period
    plt.plot(forecast_results['test'].index, forecast_results['forecast'], label='Forecast', color='red')
    
    # Plot future forecast
    future_index = pd.date_range(start=ts_data.index[-1], periods=4, freq='M')[1:]
    plt.plot(future_index, forecast_results['future_forecast'], label='Future Forecast', color='green', linestyle='--')
    
    plt.title(f"ARIMA Forecast for Product {sample_product_id} at Store {sample_store_id}")
    plt.xlabel("Month")
    plt.ylabel("Units Sold")
    plt.legend()
    plt.tight_layout()
    plt.savefig("arima_forecast.png")
    plt.close()
else:
    print("Not enough data points for ARIMA modeling.")

## 6. Inventory Optimization Recommendations

Based on our forecasts, we'll generate inventory optimization recommendations.

In [ ]:
# Function to generate inventory recommendations based on forecasts
def generate_inventory_recommendations(store_id, product_id, forecast_data, lead_time_days=7, safety_factor=1.5):
    # Get product details
    product_query = f"""
    SELECT p.*, i.quantity_on_hand, i.quantity_reserved
    FROM solving_inventory.Products p
    JOIN solving_inventory.Inventory i ON p.product_id = i.product_id
    WHERE p.product_id = '{product_id}' AND i.store_id = '{store_id}'
    """
    
    product_details = pd.read_sql(product_query, engine)
    
    if product_details.empty:
        return "Product or store not found."
    
    # Extract relevant details
    current_stock = product_details.iloc[0]['quantity_on_hand']
    reserved_stock = product_details.iloc[0]['quantity_reserved']
    available_stock = current_stock - reserved_stock
    unit_cost = product_details.iloc[0]['unit_cost']
    
    # Calculate average monthly forecast
    avg_monthly_forecast = forecast_data['future_forecast'].mean()
    
    # Convert to daily forecast
    avg_daily_forecast = avg_monthly_forecast / 30
    
    # Calculate optimal reorder point
    reorder_point = int((avg_daily_forecast * lead_time_days) + (safety_factor * avg_daily_forecast * np.sqrt(lead_time_days)))
    
    # Calculate optimal order quantity (EOQ formula simplified)
    annual_demand = avg_daily_forecast * 365
    holding_cost_percent = 0.25  # Assume 25% annual holding cost
    ordering_cost = 50  # Assume $50 per order
    
    eoq = int(np.sqrt((2 * annual_demand * ordering_cost) / (unit_cost * holding_cost_percent)))
    
    # Generate recommendations
    recommendations = {
        'store_id': store_id,
        'product_id': product_id,
        'current_stock': current_stock,
        'available_stock': available_stock,
        'avg_monthly_forecast': avg_monthly_forecast,
        'avg_daily_forecast': avg_daily_forecast,
        'optimal_reorder_point': reorder_point,
        'economic_order_quantity': eoq,
        'days_of_supply': int(available_stock / avg_daily_forecast) if avg_daily_forecast > 0 else float('inf'),
        'recommendation': ''
    }
    
    # Determine recommendation
    if available_stock <= reorder_point:
        recommendations['recommendation'] = f"REORDER NEEDED: Place order for {eoq} units. Current stock will last approximately {recommendations['days_of_supply']} days."
    elif available_stock > (reorder_point * 3):
        recommendations['recommendation'] = f"POTENTIAL OVERSTOCK: Current stock exceeds 3x reorder point. Consider reducing future orders or redistributing {int(available_stock - (reorder_point * 1.5))} units to other locations."
    else:
        recommendations['recommendation'] = f"STOCK LEVEL ADEQUATE: Current stock will last approximately {recommendations['days_of_supply']} days. Next review in {int(recommendations['days_of_supply'] - lead_time_days)} days."
    
    return recommendations

# Generate recommendations for our sample product
if 'forecast_results' in locals():
    recommendations = generate_inventory_recommendations(sample_store_id, sample_product_id, forecast_results)
    
    print("\nInventory Optimization Recommendations:")
    for key, value in recommendations.items():
        print(f"{key}: {value}")
else:
    print("Forecast results not available for generating recommendations.")

## 7. Batch Processing for Multiple Products

We'll extend our analysis to process multiple products and generate a comprehensive report.

In [ ]:
# Function to process multiple products
def batch_process_products(product_list, max_products=5):
    results = []
    
    # Limit the number of products to process
    products_to_process = product_list.head(max_products)
    
    for _, row in products_to_process.iterrows():
        store_id = row['store_id']
        product_id = row['product_id']
        
        print(f"Processing product {product_id} at store {store_id}...")
        
        # Prepare time series
        ts_data = prepare_time_series(store_id, product_id)
        
        # Skip if not enough data
        if len(ts_data) <= 5:
            print(f"  Skipping due to insufficient data ({len(ts_data)} data points).")
            continue
        
        try:
            # Fit ARIMA model
            forecast_results = fit_arima_and_forecast(ts_data)
            
            # Generate recommendations
            recommendations = generate_inventory_recommendations(store_id, product_id, forecast_results)
            
            # Add to results
            results.append(recommendations)
            
            print(f"  Processed successfully. Recommendation: {recommendations['recommendation'][:50]}...")
        except Exception as e:
            print(f"  Error processing: {str(e)}")
    
    return pd.DataFrame(results)

# Process top products
print("Batch processing top products...")
recommendations_df = batch_process_products(top_products)

# Display results
if not recommendations_df.empty:
    print("\nSummary of Recommendations:")
    print(recommendations_df[['product_id', 'current_stock', 'optimal_reorder_point', 'economic_order_quantity']])
    
    # Save to CSV
    recommendations_df.to_csv("inventory_recommendations.csv", index=False)
    print("\nRecommendations saved to 'inventory_recommendations.csv'")
else:
    print("No recommendations generated.")

## 8. Integration with SQL Database

Finally, we'll write our forecasts and recommendations back to the database for use in dashboards.

In [ ]:
# Function to write forecasts back to database
def write_forecasts_to_database(recommendations_df):
    if recommendations_df.empty:
        return "No data to write."
    
    try:
        # Create a temporary table for forecasts if it doesn't exist
        create_table_query = """
        CREATE TABLE IF NOT EXISTS solving_inventory.product_forecasts (
            forecast_id SERIAL PRIMARY KEY,
            store_id VARCHAR(50) NOT NULL,
            product_id VARCHAR(50) NOT NULL,
            forecast_date DATE NOT NULL DEFAULT CURRENT_DATE,
            avg_monthly_forecast DECIMAL(10,2),
            avg_daily_forecast DECIMAL(10,2),
            optimal_reorder_point INT,
            economic_order_quantity INT,
            days_of_supply INT,
            recommendation TEXT,
            CONSTRAINT uq_forecast_store_product UNIQUE (store_id, product_id, forecast_date)
        );
        """
        
        with engine.connect() as connection:
            connection.execute(create_table_query)
        
        # Write the data to the database
        recommendations_df.to_sql('product_forecasts', engine, schema='solving_inventory', 
                                 if_exists='append', index=False,
                                 method='multi', chunksize=100)
        
        return f"Successfully wrote {len(recommendations_df)} forecasts to database."
    except Exception as e:
        return f"Error writing to database: {str(e)}"

# Write our recommendations to the database
if 'recommendations_df' in locals() and not recommendations_df.empty:
    # Select only the columns we want to write
    forecast_columns = ['store_id', 'product_id', 'avg_monthly_forecast', 'avg_daily_forecast', 
                        'optimal_reorder_point', 'economic_order_quantity', 'days_of_supply', 'recommendation']
    
    forecasts_to_write = recommendations_df[forecast_columns]
    
    result = write_forecasts_to_database(forecasts_to_write)
    print(result)
else:
    print("No recommendations available to write to database.")

## 9. Conclusion

This notebook demonstrates how Python forecasting models can be integrated with SQL data to provide predictive inventory insights. The approach combines:

1. **SQL Views** for efficient data extraction and KPI calculation
2. **Time Series Analysis** to understand sales patterns and seasonality
3. **ARIMA Forecasting** to predict future demand
4. **Inventory Optimization** to calculate reorder points and economic order quantities
5. **Database Integration** to make forecasts available for dashboards

By implementing this solution, Urban Retail Co. can move from reactive to proactive inventory management, reducing stockouts and overstock situations while optimizing working capital.